In [1]:
from deep_dating.networks import DatingCNN
from deep_dating.datasets import DatingDataLoader, DatasetName, SetType
from deep_dating.metrics import DatingMetrics
from deep_dating.preprocessing import PreprocessRunner
from deep_dating.util import SEED, save_figure
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle

Seed set to 43


In [29]:
def merge_patches(all_labels, all_outputs, all_paths, p=None):

    preds = {}

    for i, img_name in enumerate(all_paths):
        img_name = PreprocessRunner.get_base_img_name(img_name)
        if p and img_name not in p:
            continue
        if not img_name in preds:
            preds[img_name] = [all_labels[i], [all_outputs[i]]]
        else:
            preds[img_name][1].append(all_outputs[i])

    features = []
    labels = []

    if p is None:
        p = preds

    for key, val in p.items():
        preds[key][1] = np.mean(preds[key][1], axis=0)
        features.append(preds[key][1])
        labels.append(preds[key][0])
        # print(preds[key][1].shape)
        # exit()
    features = np.array(features)

    return np.array(labels), features, preds

In [30]:
with open("runs/Feb15-19-21-40/model_epoch_0_feats_train.pkl", "rb") as f:
    labels_train_low, features_train_low, all_paths_train_low = pickle.load(f)
    labels_train_low = labels_train_low.flatten()
    labels_train_low, features_train_low, train_p = merge_patches(labels_train_low, features_train_low, all_paths_train_low)

with open("runs/Feb15-19-21-40/model_epoch_0_feats_val.pkl", "rb") as f:
    labels_val_low, features_val_low, all_paths_val_low = pickle.load(f)
    labels_val_low = labels_val_low.flatten()
    labels_val_low, features_val_low, val_p = merge_patches(labels_val_low, features_val_low, all_paths_val_low)

with open("runs/Feb18-10-52-17/model_epoch_1_feats_train.pkl", "rb") as f:
    labels_train_high, features_train_high, all_paths_train_high = pickle.load(f)
    labels_train_high = labels_train_high.flatten()
    labels_train_high, features_train_high, _ = merge_patches(labels_train_high, features_train_high, all_paths_train_high, train_p)

with open("runs/Feb18-10-52-17/model_epoch_1_feats_val.pkl", "rb") as f:
    labels_val_high, features_val_high, all_paths_val_high = pickle.load(f)
    labels_val_high = labels_val_high.flatten()
    labels_val_high, features_val_high, _ = merge_patches(labels_val_high, features_val_high, all_paths_val_high, val_p)

In [31]:
features_train_high.shape

(2692, 1536)

In [32]:
features_train_low.shape

(2692, 1536)

In [64]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import keras
from keras import layers
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder


scaler = StandardScaler()

In [78]:
new_train_x = np.hstack([features_train_low])
new_val_x = np.hstack([features_val_low])

In [54]:
svm = RandomForestClassifier(random_state=43) #MLPClassifier(batch_size=32, solver="adam", hidden_layer_sizes=(1536, 1000, 512), verbose=True, early_stopping=True, n_iter_no_change=5)

svm.fit(new_train_x, labels_train_low)
labels_val_low_predict = svm.predict(new_val_x)


In [79]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(labels_train_low)
y_train = to_categorical(y_train)
y_val = label_encoder.transform(labels_val_low)
y_val = to_categorical(y_val)
print(y_train.shape)

(2692, 15)


In [81]:
model = keras.Sequential(
    [
        keras.Input(shape=(1536)),
        layers.Dense(1024, activation="relu"),
        layers.Dense(512, activation="relu"),
        layers.Dense(15, activation="softmax"),
    ]
)
batch_size = 32
epochs = 100

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.00001), metrics=["accuracy"])
model.fit(new_train_x, y_train, batch_size=batch_size, epochs=epochs, validation_data=(new_val_x, y_val))


Epoch 1/100
85/85 [==============================] - 2s 14ms/step - loss: 3.5272 - accuracy: 0.1542 - val_loss: 2.4462 - val_accuracy: 0.1751
Epoch 2/100
85/85 [==============================] - 1s 13ms/step - loss: 2.4482 - accuracy: 0.2515 - val_loss: 2.3174 - val_accuracy: 0.3644
Epoch 3/100
85/85 [==============================] - 1s 13ms/step - loss: 2.4117 - accuracy: 0.2771 - val_loss: 2.2727 - val_accuracy: 0.3743
Epoch 4/100
85/85 [==============================] - 1s 13ms/step - loss: 2.3613 - accuracy: 0.3124 - val_loss: 2.1708 - val_accuracy: 0.4477
Epoch 5/100
85/85 [==============================] - 1s 13ms/step - loss: 2.3051 - accuracy: 0.3782 - val_loss: 2.2229 - val_accuracy: 0.3828
Epoch 6/100
85/85 [==============================] - 1s 13ms/step - loss: 2.2914 - accuracy: 0.3533 - val_loss: 2.1613 - val_accuracy: 0.2472
Epoch 7/100
85/85 [==============================] - 1s 13ms/step - loss: 2.2219 - accuracy: 0.3614 - val_loss: 2.0436 - val_accuracy: 0.4703
Epoch 

In [55]:
alphas = [0, 25, 50]
metrics = DatingMetrics(alphas=alphas)
metrics.names

vals = metrics.calc(labels_val_low, labels_val_low_predict)

mae, mse = tuple(vals[:2])
cs_ = vals[2:]

print(mae, mse)
print([0, 25])
print(cs_)

# 19.00423728813559 1980.247175141243
# [0, 25]
# [64.97175141242938, 80.64971751412429, 91.66666666666666]

# 19.326271186440678 2154.9138418079096
# [0, 25]
# [65.11299435028248, 80.9322033898305, 91.38418079096046]

19.307909604519775 2265.762711864407
[0, 25]
[64.83050847457628, 80.7909604519774, 92.37288135593221]
